##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/R/R06_saving_and_loading_models.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/rses-dl-course/rses-dl-course.github.io/blob/master/notebooks/R/R06_saving_and_loading_models.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Lab 06: Saving and Loading Models

In this lab we will learn how we can take a trained model, save it, and then load it back to keep training it or use it to perform inference. In particular, we will use transfer learning to train a classifier to classify images of cats and dogs, just like we did in the previous lesson. We will then take our trained model and save it as an HDF5 file, which is the format used by Keras. We will then load this model, use it to perform predictions, and then continue to train the model. Finally, we will save our trained model as a TensorFlow SavedModel and then we will download it to a local disk, so that it can later be used for deployment in different platforms.

## Concepts that will be covered in this Colab

1. Saving models in HDF5 format for Keras
3. Loading models

Before starting this Colab, you should reset the Colab environment by selecting `Runtime -> Reset all runtimes...` from menu above.

# Install and load dependencies


First, you'll need to install and load R package Keras which will also install TensorFlow. We'll also install package fs which has useful functionality for working with our filesystem.

In [ ]:
install.packages(c("keras", "fs"))
library(keras)

# Part 1: Load the Cats vs. Dogs Dataset

## Dataset

We download the dataset again. The dataset we are using is a filtered version of <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs. Cats</a> dataset from Kaggle (ultimately, this dataset is provided by Microsoft Research).

In [ ]:
URL <- "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_dir <- get_file('cats_and_dogs_filterted.zip', origin = URL, extract = TRUE)

The dataset we have downloaded has the following directory structure.

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ...]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ...]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

We can list the directories with the following terminal command:

In [ ]:
zip_dir_base <- dirname(zip_dir)
fs::dir_tree(zip_dir_base, recurse = 2)

We'll now assign variables with the proper file path for the training and validation sets. We'll also create some variables that hold information about the size of our datasets

In [ ]:
base_dir <- fs::path(zip_dir_base, "cats_and_dogs_filtered")
train_dir <- fs::path(base_dir, "train")
validation_dir <- fs::path(base_dir, "validation")

train_cats_dir <- fs::path(train_dir, "cats")
train_dogs_dir <- fs::path(train_dir, "dogs")
validation_cats_dir <- fs::path(validation_dir, "cats")
validation_dogs_dir <- fs::path(validation_dir, "dogs")


num_cats_tr <- length(fs::dir_ls(train_cats_dir))
num_dogs_tr <- length(fs::dir_ls(train_dogs_dir))

num_cats_val <- length(fs::dir_ls(validation_cats_dir))
num_dogs_val <- length(fs::dir_ls(validation_dogs_dir))

total_train <- num_cats_tr + num_dogs_tr
total_val <- num_cats_val + num_dogs_val

Lets create training and validation image generators to read our images from their directories which rescales our image to values from 0 to 1. Let's also create a flow from our training directories which also resizes our images to the resolution (224 x 224) expected by our MobileNetV2 model we'll be using for transfer learning. Let's also set the batch size to 32.

In [ ]:
batch_size <- 32
image_res <- 224

# training generators
train_image_generator <- image_data_generator(rescale = 1/255,
                                              rotation_range = 45,
                                              width_shift_range = 0.2,
                                              height_shift_range = 0.2,
                                              shear_range = 0.2,
                                              zoom_range = 0.2,
                                              horizontal_flip = TRUE,
                                              fill_mode = 'nearest')

train_data_gen <- flow_images_from_directory(directory = train_dir,
                                             generator = train_image_generator,
                                             target_size = c(image_res, image_res),
                                             class_mode = "binary",
                                             batch_size = batch_size)

# validation generators
val_image_generator <- image_data_generator(rescale = 1/255)

val_data_gen <- flow_images_from_directory(directory = validation_dir,
                                             generator = val_image_generator,
                                             target_size = c(image_res, image_res),
                                             class_mode = "binary",
                                             batch_size = batch_size)

# Part 2: Transfer Learning with TensorFlow Hub


Let's now use the [MobileNet V2 model architecture](https://keras.rstudio.com/reference/application_mobilenet_v2.html) to do Transfer Learning.

In [ ]:
feature_extractor <- application_mobilenet_v2(input_shape = c(image_res, image_res, 3),
                                              include_top = FALSE, pooling = "avg")

Freeze the variables in the feature extractor layer, so that the training only modifies the final classifier layer.

In [ ]:
freeze_weights(feature_extractor)

Now let's use are feature extractor as part of a keras sequential model, and add a new classification layer of 2 units with softmax activation. 

In [ ]:
model <- keras_model_sequential() %>%
          feature_extractor() %>%
          layer_dense(units = 2, activation = "softmax")
model

## Train the model

We now train this model like any other using an image generator, by first calling `compile` followed by `fit_generator`. 

In [ ]:
model %>% compile(optimizer="adam",
                  loss = loss_sparse_categorical_crossentropy,
                  metrics = "accuracy")

epochs = 3
history <- model %>%
              fit_generator(generator = train_data_gen,
              steps_per_epoch = as.integer(total_train / batch_size),
              epochs = epochs,
              validation_data = val_data_gen,
              validation_steps = as.integer(total_val / batch_size))

cat('Validation loss:', format(tail(history$metrics$val_loss, 1), digits = 2), "\n")
cat('Validation accuracy:', format(tail(history$metrics$val_accuracy, 1), digits = 2), "\n")

## Check the predictions

To check our model, let's use it to make some predictions on the images from a single batch of our validation image generator. 

Let's compile our predictions into a data.frame. Let's also add the actual labels for each row to the data.frame and have a look at our predictions. We can see that predictions using our transfer learning very accurate with very high confidence in each prediciton!

In [ ]:
pred_batch <- val_data_gen[1]

pred_mat <- model %>%
  predict(pred_batch[[1]]) 

predictions <- data.frame(class_description = names(val_data_gen$class_indices)[apply(pred_mat, 1, which.max)],
                          score = apply(pred_mat, 1, max),
                          label = names(val_data_gen$class_indices)[pred_batch[[2]] + 1])
predictions

Let's now plot the images from our Dogs vs Cats dataset and put the predicted labels above them and their actual labels below.  First we create a plotting function:

In [ ]:
plot_rgb_image_ttl <- function(image_array, prediction){
  image_array %>%
  array_reshape(dim = c(dim(.)[1:3])) %>%
  as.raster(max = 1) %>%
  plot()
  title(main = paste0(prediction$class_description, 
                      " (", format(prediction$score * 100, digits = 2), "%)"),
        sub = prediction$label)
}

Then we loop the function over the first 10 images:

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

# Set the layout
layout(matrix(1:10, ncol = 5))

# Loop plotting over the batch of images
for(i in 1:10){
  plot_rgb_image_ttl(pred_batch[[1]][i,,,], predictions[i,])
  }

# Part 3: Save as Keras `.h5` model

Now that we've trained the model,  we can save it as an HDF5 file, which is the format used by Keras using function [`save_model_hdf5()`](https://keras.rstudio.com/reference/save_model_hdf5.html). Our HDF5 file will have the extension '.h5', and it's name will correpond to the current time stamp.

In [ ]:
export_path_keras = paste0("./", as.integer(Sys.time()) ,".h5")
export_path_keras

In [ ]:
model %>% save_model_hdf5(filepath = export_path_keras)


In [ ]:
fs::dir_tree(recurse = 0)

 You can later recreate the same model from this file, even if you no longer have access to the code that created the model.

This file includes:

- The model's architecture
- The model's weight values (which were learned during training)
- The model's training config (what you passed to `compile`), if any
- The optimizer and its state, if any (this enables you to restart training where you left off)

# Part 4:  Load the Keras `.h5` Model

We will now load the model we just saved into a new model called `reloaded`. 

In [ ]:
reloaded_model <- load_model_hdf5(export_path_keras, 
                                  custom_objects = list(loss_sparse_categorical_crossentropy = loss_sparse_categorical_crossentropy))
                    
reloaded_model

We can check that the reloaded model and the previous model give the same result. Let's make some predictions using the reloaded model on the batch of images we had previously used.

In [ ]:
pred_mat_reloaded <- reloaded_model %>%
  predict(pred_batch[[1]]) 

Predicted classes from both models should be equal so the following statement should return `TRUE`

In [ ]:
all(apply(pred_mat, 1, which.max) == apply(pred_mat_reloaded, 1, which.max))

# Keep Training

Besides making predictions, we can also take our `reloaded_model` and keep training it. To do this, you can just train the `reloaded_model` as usual, using the `fit_generator` method.

In [ ]:
epochs = 3
history <- reloaded_model %>%
              fit_generator(generator = train_data_gen,
              steps_per_epoch = as.integer(total_train / batch_size),
              epochs = epochs,
              validation_data = val_data_gen,
              validation_steps = as.integer(total_val / batch_size))

cat('Validation loss:', format(tail(history$metrics$val_loss, 1), digits = 2), "\n")
cat('Validation accuracy:', format(tail(history$metrics$val_accuracy, 1), digits = 2), "\n")